In [2]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader, Dataset
import RNA
import json
import pandas as pd
import forgi.visual.mplotlib as fvm
import forgi
import matplotlib.pyplot as plt
import webbrowser
import numpy as np

In [3]:
train = pd.read_json('stanford-covid-vaccine/train.json', lines=True)
idx = 20
indices = [i for i in range(len(train.iloc[idx]['structure']))]
print(f"Sequence: {train.iloc[idx]['sequence']}\nStructure: {train.iloc[idx]['structure']}\nIndex: {indices}")

Sequence: GGAAAUCUAGCGACCCAGGGACGGAAGUCAACUAGACGAAGUAGAUGCGAUCAGAGUUUGCGUUACAAGCGCCUCUUCGGAGGUGCAAAAGAAACAACAACAACAAC
Structure: .....(((((.(((((......))..)))..)))))....((.(((((((.......)))))))))..(((((((....))))))).....................
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106]


In [4]:

def check(seq, struc):
    i = 0
    while True:
        i+=1
        if struc[i] == ')':
            for x in range(len(struc[i:])):
                idx = i+x
                if struc[i+x] == '(':
                    return idx, struc[:idx], struc
                    break


def get_all_portions(seq, struc):
    a = []
    try:
        copy = struc
        while True:
            idx, new, _ = check(seq, copy)
            a.append(new)
            copy = copy[idx:]
    except:
        a.append(copy)

    return a


In [5]:

def visualize(seq, struc):
    viz = f'http://nibiru.tbi.univie.ac.at/forna/forna.html?id=fasta&file=%3Eheader%5Cn{seq}%5Cn{struc}'
    webbrowser.open(viz)

def clean_ends(splits):
    all_len = len(splits)
    copy_splits = splits
    for idx, split in enumerate(splits):
        first = split.find('(')
        last = split[::-1].find(')')
        copy = split
        copy = copy[::-1][last:]
        copy = copy[::-1][first:]
        copy_splits[idx] = copy
    return copy_splits[:all_len]
        

eg_seq = train.iloc[idx]['sequence']
eg_struc = train.iloc[idx]['structure']

# visualize(eg_seq, eg_struc)


new_splits = clean_ends(get_all_portions(eg_seq, eg_struc))
print(eg_struc)
print(new_splits)


.....(((((.(((((......))..)))..)))))....((.(((((((.......)))))))))..(((((((....))))))).....................
['(((((.(((((......))..)))..)))))', '((.(((((((.......)))))))))', '(((((((....)))))))']


In [6]:
def base_pairing(any_splits):
    all_things = []
    for idx, split in enumerate(any_splits):
        all_open = [x for x, g in enumerate(split) if g=='(']
        all_close = [x for x, g in enumerate(split) if g==')']
        middle_bracket = [max(all_open), min(all_close)]
        all_things.append(middle_bracket)
    return all_things

def find_bonds(seq, struc):
    splits = clean_ends(get_all_portions(seq, struc))
    middles = base_pairing(splits)
    
    for idx, middle in enumerate(middles):
        padding_left=int(struc.find(splits[idx]))
        new_vals = [middle[0]+padding_left, middle[1]+padding_left]
        middles[idx] = new_vals

    return middles, struc, splits


def test_pair(seq, struc, idx_type=0):
    bondage = []
    middles, copy_struc, splits = find_bonds(seq, struc)
    left, right = middles[idx_type]
    while True:
        try:
            if copy_struc[left] == '(' and copy_struc[right] == ')':
                bondage.append([left, right])
                left-=1
                right+=1
            elif copy_struc[left] == '(' and copy_struc[right] == '.':
                right +=1
            elif copy_struc[left] == '.' and copy_struc[right] == ')':
                left -=1
            elif copy_struc[left] == '.' and copy_struc[right] == '.':
                left -=1
                right +=1
            else:
                break
        except:
            break

    return middles, copy_struc, splits, bondage


In [26]:
def get_sequence_bonds(idx):

    eg_seq = train.iloc[idx]['sequence']
    eg_struc = train.iloc[idx]['structure']

    symmetric_bonds = []
    repl_struc = ''


    for i in range(len(clean_ends(get_all_portions(eg_seq, eg_struc)))):
        eg_middles, repl_struc, eg_splits, eg_bondage = test_pair(eg_seq, eg_struc, i)
        symmetric_bonds += eg_bondage



    try:
        while ('(' or ')') in repl_struc:
            split_struc = [i for i in repl_struc]

            for coord in symmetric_bonds:
                l, r = coord
                split_struc[l] = '.'
                split_struc[r] = '.'

            repl_struc = ''.join(split_struc)
            _, _, _, final_bondage = test_pair(eg_seq, repl_struc)
            symmetric_bonds += final_bondage
    except:
        pass

    all_bonds = [list(x) for x in set(tuple(x) for x in symmetric_bonds)]

    # print(eg_struc)
    # print(repl_struc)
    # print(len(all_bonds))
    # print(len([char for char in eg_struc if char == '(']))
    return all_bonds, eg_struc, eg_seq


In [27]:
dataset_adjacency = {key: [] for key in range(len(train))}

for idx in range(len(train)):
    flag = ''
    eg_struc = train.iloc[idx]['structure']
    all_bonds, check_struc, check_seq = get_sequence_bonds(idx)
    if len(all_bonds) != len([char for char in eg_struc if char == '(']):
        flag = 'hmm sus'
    else:
        dataset_adjacency[idx] = [check_seq, check_struc, all_bonds]
        flag = 'All structures are accounted for'        


In [28]:
print(dataset_adjacency[0][2])
visualize(dataset_adjacency[0][0], str(dataset_adjacency[0][1]))


[[27, 62], [74, 79], [26, 63], [10, 18], [34, 55], [72, 81], [42, 47], [73, 80], [6, 23], [41, 48], [33, 56], [40, 49], [9, 19], [5, 24], [39, 50], [8, 20], [71, 82], [7, 21], [69, 84], [70, 83], [38, 51], [37, 52], [68, 85]]


In [124]:
# indices = [i for i in range(len(eg_struc))]
# matr = np.zeros((len(indices), len(indices)))

# def watson_crick(struc):
#     fivethree = struc
#     threefive = struc[::-1]

#     for idx in range(len(struc)):
#         i = idx
#         if fivethree[idx] == '(':
#             while True:
#                 if threefive[i] == ')':
#                     return i, idx
#                     break
#                 i+=1
            
#             break


# def all_pairs(seq, struc):
#     bonds = []
#     copy = struc
#     sp = [char for char in copy]
#     x = sp.count('(')
#     x*=2
#     while len(bonds) < x:
#         try:
#             a = watson_crick(copy)
#             a = list(a)
#             tf_idx = a[0]
#             ft_idx = a[1]
#             bonds.append(ft_idx)
#             bonds.append(tf_idx)
#             copy = copy[::-1]
#             copy = copy[tf_idx+1:]
#             copy = copy[::-1]
#             copy = copy[ft_idx+1:]
#         except:
#             break

#     return bonds

    
# all_bonds = all_pairs(eg_seq, eg_struc)

In [66]:
a = '.....(((((.(((((......))..)))..)))))....((.(((((((.......)))))))))..(((((((....))))))).....................'
a.find('(((((.(((((......))..)))..)))))....((.(((((((.......)))))))))..(((((((....))))))).....................')


5